#Advanced Net (transformer)

#Розділення даних на набори

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Завантаження даних
data = pd.read_csv("small_data.csv")

# Розділення на тренувальний, валідаційний та тестовий набори
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)  # 80% на тренування
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)  # 10% валідація, 10% тест

# Збереження розділених наборів
train_data.to_csv("train_data.csv", index=False)
val_data.to_csv("val_data.csv", index=False)
test_data.to_csv("test_data.csv", index=False)

print("Дані успішно розділено та збережено.")

Дані успішно розділено та збережено.


In [ ]:
import pandas as pd

# Завантаження частини даних
train_data = pd.read_csv("train_data_tokenized.csv", nrows=5)
print(train_data.head())
print("Розмір даних:", train_data.memory_usage(deep=True).sum(), "байт")

                                             English  \
0  Visit a two-storey bathhouse with a thermae fo...   
1  Therefore, it is widely used as an amplifier w...   
2  Our clients abroad include both world-leading ...   
3  And I did not believe them that told me, till ...   
4                                 seizing mechanism.   

                                           Ukrainian  \
0  Відвідайте двоповерхову лазню з парною на 8 ос...   
1  Тому він широко використовується як діюча підс...   
2  Наші клієнти за кордоном - це як світові лідер...   
3  7 І не повірила я тим словам, аж поки не прийш...   
4                                заїдання механізму.   

                                        uk_tokenized  \
0  [382, 22156, 4480, 12175, 37, 8, 8590, 960, 22...   
1  [8, 1848, 324, 6465, 8, 3594, 123, 15114, 28, ...   
2  [8, 5740, 19802, 62, 13060, 25, 203, 123, 5602...   
3  [535, 308, 42, 16315, 1720, 378, 749, 1312, 96...   
4                      [62, 209, 8205, 3657, 3

In [ ]:
print("Максимальна довжина українських токенів:", train_data['uk_tokenized'].apply(len).max())
print("Мінімальна довжина українських токенів:", train_data['uk_tokenized'].apply(len).min())

print("Максимальна довжина англійських токенів:", train_data['en_tokenized'].apply(len).max())
print("Мінімальна довжина англійських токенів:", train_data['en_tokenized'].apply(len).min())

Максимальна довжина українських токенів: 241
Мінімальна довжина українських токенів: 29
Максимальна довжина англійських токенів: 239
Мінімальна довжина англійських токенів: 23


#Токенізація

In [ ]:
pip install sentencepiece transformers

In [ ]:
#Створення токенізатора за допомогою SentencePiece
import sentencepiece as spm

# Створення токенізатора
spm.SentencePieceTrainer.train(
    input='train_data.csv',  # Вхідний файл
    model_prefix='tokenizer',  # Префікс для моделі
    vocab_size=32000,  # Розмір словника
    pad_id=0, unk_id=1, bos_id=2, eos_id=3,  # Спеціальні токени
    user_defined_symbols=['<sep>', '<cls>']  # Додаткові символи (за потреби)
)

print("Токенізатор створено.")

Токенізатор створено.


Це створить два файли: tokenizer.model і tokenizer.vocab.

In [ ]:
import sentencepiece as spm

# Завантаження токенізатора
sp = spm.SentencePieceProcessor()
sp.load("tokenizer.model")

True

In [ ]:
# Токенізація українських текстів
train_data['uk_tokenized'] = train_data['Ukrainian'].apply(lambda x: sp.encode(x, out_type=int))

# Токенізація англійських текстів
train_data['en_tokenized'] = train_data['English'].apply(lambda x: sp.encode(x, out_type=int))

# Перевіримо результат
print(train_data[['uk_tokenized', 'en_tokenized']].head())

                                        uk_tokenized  \
0  [382, 22156, 4480, 12175, 37, 8, 8590, 960, 22...   
1  [8, 1848, 324, 6465, 8, 3594, 123, 15114, 28, ...   
2  [8, 5740, 19802, 62, 13060, 25, 203, 123, 5602...   
3  [535, 308, 42, 16315, 1720, 378, 749, 1312, 96...   
4                      [62, 209, 8205, 3657, 37, 19]   

                                        en_tokenized  
0  [14250, 23, 477, 18, 17758, 5311, 7979, 54, 23...  
1  [11952, 6, 72, 29, 12128, 683, 105, 119, 30998...  
2  [2577, 5466, 7779, 2830, 1375, 712, 18, 8265, ...  
3  [394, 155, 1369, 61, 3032, 313, 70, 4719, 572,...  
4                            [3538, 5702, 11146, 19]  


In [ ]:
train_data.to_csv("train_data_tokenized.csv", index=False)
print("Токенізовані дані збережено у train_data_tokenized.csv")

Токенізовані дані збережено у train_data_tokenized.csv


#Імплементація Transformer

In [ ]:
pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Embedding, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Add

# Параметри моделі
embed_dim = 512  # Розмір ембеддінгів
num_heads = 8    # Кількість голов у MultiHeadAttention
ff_dim = 2048    # Розмір feed-forward шару
vocab_size = 32000  # Розмір словника (українського та англійського)

# Енкодер
def transformer_encoder(inputs, embed_dim, num_heads, ff_dim, dropout=0.1):
    attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(inputs, inputs)
    attention = Dropout(dropout)(attention)
    attention = Add()([inputs, attention])
    attention = LayerNormalization(epsilon=1e-6)(attention)

    ff = Dense(ff_dim, activation="relu")(attention)
    ff = Dense(embed_dim)(ff)
    ff = Dropout(dropout)(ff)
    ff = Add()([attention, ff])
    return LayerNormalization(epsilon=1e-6)(ff)

# Декодер
def transformer_decoder(inputs, encoder_output, embed_dim, num_heads, ff_dim, dropout=0.1):
    attention1 = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(inputs, inputs)
    attention1 = Dropout(dropout)(attention1)
    attention1 = Add()([inputs, attention1])
    attention1 = LayerNormalization(epsilon=1e-6)(attention1)

    attention2 = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(attention1, encoder_output)
    attention2 = Dropout(dropout)(attention2)
    attention2 = Add()([attention1, attention2])
    attention2 = LayerNormalization(epsilon=1e-6)(attention2)

    ff = Dense(ff_dim, activation="relu")(attention2)
    ff = Dense(embed_dim)(ff)
    ff = Dropout(dropout)(ff)
    ff = Add()([attention2, ff])
    return LayerNormalization(epsilon=1e-6)(ff)

# Побудова моделі
encoder_inputs = Input(shape=(None,), dtype="int32", name="encoder_inputs")
x = Embedding(vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = transformer_encoder(x, embed_dim, num_heads, ff_dim)

decoder_inputs = Input(shape=(None,), dtype="int32", name="decoder_inputs")
x = Embedding(vocab_size, embed_dim)(decoder_inputs)
x = transformer_decoder(x, encoder_outputs, embed_dim, num_heads, ff_dim)

outputs = Dense(vocab_size, activation="softmax")(x)

model = Model([encoder_inputs, decoder_inputs], outputs)

# Компіляція моделі
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 embedding (Embedding)       (None, None, 512)            1638400   ['encoder_inputs[0][0]']      
                                                          0                                       
                                                                                                  
 multi_head_attention (Mult  (None, None, 512)            8401408   ['embedding[0][0]',           
 iHeadAttention)                                                     'embedding[0][0]']       

In [ ]:
# Завантаження валідаційного набору
val_data = pd.read_csv("val_data.csv")

# Токенізація українських текстів
val_data['uk_tokenized'] = val_data['Ukrainian'].apply(lambda x: sp.encode(x, out_type=int))

# Токенізація англійських текстів
val_data['en_tokenized'] = val_data['English'].apply(lambda x: sp.encode(x, out_type=int))

# Збереження токенізованого валідаційного набору
val_data.to_csv("val_data_tokenized.csv", index=False)
print("Файл val_data_tokenized.csv створено.")

Файл val_data_tokenized.csv створено.


In [ ]:
val_data = pd.read_csv("/content/val_data_tokenized.csv")

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Визначте максимальну довжину послідовностей
max_length = 100  # Наприклад, обмежимо до 100 токенів

# Padding і truncating для тренувального набору
X_train = pad_sequences(train_data['uk_tokenized'], maxlen=max_length, padding='post', truncating='post')
Y_train = pad_sequences(train_data['en_tokenized'], maxlen=max_length, padding='post', truncating='post')

print("Форма X_train:", X_train.shape)
print("Форма Y_train:", Y_train.shape)

Форма X_train: (5, 100)
Форма Y_train: (5, 100)


In [ ]:
print(val_data['uk_tokenized'].head())

0    [9113, 154, 56, 2910, 95, 8, 1204, 2344, 22, 1...
1    [430, 308, 378, 8, 19326, 6, 20, 3166, 1546, 2...
2    [5167, 1014, 9, 5457, 2717, 2174, 9, 8, 17234,...
3    [14433, 545, 1013, 3497, 42, 603, 10790, 37, 1...
4    [25, 8, 8554, 13425, 18788, 27424, 42, 2411, 2...
Name: uk_tokenized, dtype: object


In [ ]:
print(type(val_data['uk_tokenized'][0]))

<class 'str'>


In [ ]:
import ast

# Перетворення українських токенів
val_data['uk_tokenized'] = val_data['uk_tokenized'].apply(ast.literal_eval)

# Перетворення англійських токенів
val_data['en_tokenized'] = val_data['en_tokenized'].apply(ast.literal_eval)

In [ ]:
print(type(val_data['uk_tokenized'][0]))

<class 'list'>


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_val = pad_sequences(val_data['uk_tokenized'], maxlen=max_length, padding='post', truncating='post')
Y_val = pad_sequences(val_data['en_tokenized'], maxlen=max_length, padding='post', truncating='post')

print("Форма X_val:", X_val.shape)
print("Форма Y_val:", Y_val.shape)

Форма X_val: (100000, 100)
Форма Y_val: (100000, 100)


#Навчання моделі Transformer

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM

# Параметри
vocab_size = 32000  # Розмір словника
embed_dim = 256  # Розмір ембеддінгів

# Енкодер
encoder_inputs = Input(shape=(None,), name='encoder_inputs')
encoder_embedding = Embedding(input_dim=vocab_size, output_dim=embed_dim, mask_zero=True)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(256, return_state=True)(encoder_embedding)

# Декодер
decoder_inputs = Input(shape=(None,), name='decoder_inputs')
decoder_embedding = Embedding(input_dim=vocab_size, output_dim=embed_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Модель
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Компіляція
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Навчання
model.fit([X_train, Y_train[:, :-1]], Y_train[:, 1:],
          validation_data=([X_val, Y_val[:, :-1]], Y_val[:, 1:]),
          batch_size=64, epochs=10)

Epoch 1/10
1/1 [==============================] - 840s 840s/step - loss: 10.3735 - accuracy: 0.0000e+00 - val_loss: 10.3729 - val_accuracy: 0.0137
Epoch 2/10
1/1 [==============================] - 820s 820s/step - loss: 10.3694 - accuracy: 0.3246 - val_loss: 10.3721 - val_accuracy: 0.0278
Epoch 3/10
1/1 [==============================] - 813s 813s/step - loss: 10.3649 - accuracy: 0.5351 - val_loss: 10.3711 - val_accuracy: 0.0373
Epoch 4/10
1/1 [==============================] - 817s 817s/step - loss: 10.3596 - accuracy: 0.5965 - val_loss: 10.3699 - val_accuracy: 0.0464
Epoch 5/10
1/1 [==============================] - 827s 827s/step - loss: 10.3528 - accuracy: 0.6053 - val_loss: 10.3680 - val_accuracy: 0.0527
Epoch 6/10
1/1 [==============================] - 829s 829s/step - loss: 10.3427 - accuracy: 0.4211 - val_loss: 10.3644 - val_accuracy: 0.0613
Epoch 7/10
1/1 [==============================] - 831s 831s/step - loss: 10.3249 - accuracy: 0.3158 - val_loss: 10.3554 - val_accuracy: 0.

In [ ]:
# Збереження моделі після навчання
model.save("transformer_model.h5")
print("Модель збережено як transformer_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Модель збережено як transformer_model.h5


In [ ]:
def translate_with_beam_search(input_sequence, beam_width=3, max_length=100):
    start_token = [2]  # <bos>
    sequences = [[start_token, 0.0]]  # Послідовності з ймовірностями

    for _ in range(max_length):
        all_candidates = []
        for seq, score in sequences:
            decoder_input = np.array([seq])
            predictions = model.predict([input_sequence, decoder_input])
            predictions = predictions[:, -1, :]  # Ймовірності для останнього токена

            # Отримуємо top-k токени
            top_k = np.argsort(predictions[0])[-beam_width:]

            # Формуємо нові послідовності
            for token in top_k:
                candidate = [seq + [int(token)], score - np.log(predictions[0][token])]
                all_candidates.append(candidate)

        # Обираємо top-k найкращих послідовностей
        sequences = sorted(all_candidates, key=lambda x: x[1])[:beam_width]

    # Повертаємо найкращу послідовність
    return sequences[0][0]

In [ ]:
input_sequence = X_val[0].reshape(1, -1)
translated_tokens = translate_with_sampling(input_sequence, temperature=0.8, top_k=5)
translated_text = sp.decode(translated_tokens)
print("Переклад:", translated_text)

1/1 [==============================] - 0s 44ms/step
Переклад: a,- with me with and with, the me with, me and the and, me, with,,, with me,, the with and, me, the me, me with and me, the, with and the me with the, with me and and with, and with the with and the me the me me, with and me and with and me, the with the with, with me with and with with and me me, the,,, and the me,,


# Схоже модель не навчилася робити переклад:(

In [ ]:
from tensorflow.keras.models import load_model

# Завантаження моделі
model = load_model("transformer_model.h5")
print("Модель успішно завантажена!")

Модель успішно завантажена!


In [ ]:
import sentencepiece as spm

# Завантаження токенізатора
sp = spm.SentencePieceProcessor()
sp.load("tokenizer.model")

True

In [ ]:
import numpy as np

def translate_text(input_text, max_length=100):
    # Токенізація вхідного тексту
    input_tokens = sp.encode(input_text, out_type=int)
    input_sequence = np.array([input_tokens])

    # Початковий токен для декодера (<bos>)
    start_token = [2]
    decoder_input = np.array([start_token])

    # Вихідна послідовність
    output_sequence = []

    for _ in range(max_length):
        # Прогнозування
        predictions = model.predict([input_sequence, decoder_input])
        predicted_token = np.argmax(predictions[:, -1, :], axis=-1)

        # Додаємо згенерований токен у вихідну послідовність
        output_sequence.append(int(predicted_token[0]))

        # Оновлення декодера
        decoder_input = np.concatenate([decoder_input, predicted_token[:, None]], axis=1)

        # Перевірка на токен <eos>
        if predicted_token[0] == 3:  # <eos>
            break

    # Декодування вихідних токенів у текст
    translated_text = sp.decode(output_sequence)
    return translated_text

In [ ]:
input_text = "Привіт, як справи?"  # Текст українською
translated_text = translate_text(input_text)
print("Переклад:", translated_text)

1/1 [==============================] - 0s 34ms/step
Переклад: -,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [ ]:
def translate_with_sampling(input_text, temperature=0.8, top_k=10, max_length=100):
    input_tokens = sp.encode(input_text, out_type=int)
    input_sequence = np.array([input_tokens])
    start_token = [2]  # <bos>
    decoder_input = np.array([start_token])
    output_sequence = []

    for _ in range(max_length):
        predictions = model.predict([input_sequence, decoder_input])
        predictions = predictions[:, -1, :] / temperature  # Регулювання температури

        # Top-k sampling
        sorted_indices = np.argsort(predictions[0])[-top_k:]
        probs = np.exp(predictions[0, sorted_indices])
        probs /= np.sum(probs)
        predicted_token = np.random.choice(sorted_indices, p=probs)

        output_sequence.append(int(predicted_token))  # Конвертація в int
        decoder_input = np.concatenate([decoder_input, [[predicted_token]]], axis=1)

        if predicted_token == 3:  # <eos>
            break

    # Переконайтеся, що output_sequence — список цілих чисел
    output_sequence = list(map(int, output_sequence))
    translated_text = sp.decode(output_sequence)
    return translated_text

In [ ]:
input_text = "Привіт, як справи?"
translated_text = translate_with_sampling(input_text, temperature=0.7, top_k=5)
print("Переклад:", translated_text)

1/1 [==============================] - 0s 35ms/step
Переклад: -- the, with me with, and and the me the me, me with and, with and me and and me with me, the and me, with, me, and and and with with, me me and with the with me and, and the, me and me me with with with with, me,, and, me,, and and and with and with, the the with the and me with and me with me,,, me, me and the with, me


#Спроба виявилася НЕВДАЛОЮ:(((. Модель навчалася більше 2 год!